In [1]:
# load a dataset directory
import os
import numpy as np
from PIL import Image

## TOF18 Dataset from Indepth Paper

In [3]:
image_dir_ToF = '/mnt/IRONWOLF1/ashkan/data/ToF-18K'

In [ ]:
for image in os.listdir(image_dir_ToF):
    if not image.endswith('_undistorted.jpg'):
        # write in the file
        print(image)
        with open('toF18_split.txt', 'a') as f:
            depth_path = os.path.join ('data', image).replace('.jpg', '_z16.png')
            f.write(os.path.join(image) + ' ' + depth_path + '\n')


## ARKitScene dataset

In [3]:
arkitScenes_path = '/mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling'

In [3]:
def gen_split_txt(split_file, th):
    for scene in os.listdir(os.path.join(arkitScenes_path, split_file)):
        for sample in os.listdir(os.path.join(arkitScenes_path, split_file, scene, 'highres_depth')):
            # if gt depth missing points are less than 20% of the image
            depth_path = os.path.join(arkitScenes_path, split_file, scene, 'highres_depth', sample)
            depth = np.array(Image.open(depth_path))
            missing = np.sum(depth == 0)
            if missing / depth.size < th:
                with open(f'arkit_split_{split_file}.txt', 'a') as f:
                    f.write(os.path.join(scene, 'lowres_wide', sample) + '\n')

In [9]:
# for scene in os.listdir(os.path.join(arkitScenes_path, 'Validation')):
#     for sample in os.listdir(os.path.join(arkitScenes_path, 'Validation', scene, 'highres_depth')):
#         # if gt depth missing points are less than 20% of the image
#         depth_path = os.path.join(arkitScenes_path, 'Validation', scene, 'highres_depth', sample)
#         depth = np.array(Image.open(depth_path))
#         missing = np.sum(depth == 0)
#         if missing / depth.size < 0.1:
#             with open(f'arkit_split_Validation.txt', 'a') as f:
#                 line = f"{os.path.join(scene, 'lowres_wide', sample)} {os.path.join(scene, 'downsampled_gt', sample)} \n"
#                 f.write(line)

In [ ]:
gen_split_txt('Training', 0.1)

In [ ]:
import threading

# Create threads for each split_file and th combination
threads = []
threads.append(threading.Thread(target=gen_split_txt, args=('Validation', 0.1)))
threads.append(threading.Thread(target=gen_split_txt, args=('Training', 0.1)))

# Start the threads
for thread in threads:
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()


In [ ]:
# for scene in os.listdir(os.path.join(arkitScenes_path, 'Validation')):
#     for sample in os.listdir(os.path.join(arkitScenes_path, 'Validation', scene, 'highres_depth')):
#         print(sample)
#         # if gt depth missing points are less than 20% of the image
#         depth_path = os.path.join(arkitScenes_path, 'Validation', scene, 'highres_depth', sample)
#         depth = np.array(Image.open(depth_path))
#         missing = np.sum(depth == 0)
#         if missing / depth.size < 0.2:
#             with open('arkit_split_validation.txt', 'a') as f:
#                 f.write(os.path.join(scene, 'lowres_wide', sample) + '\n')

In [ ]:
# for scene in os.listdir(os.path.join(arkitScenes_path, 'Training')):
#     for sample in os.listdir(os.path.join(arkitScenes_path, 'Training', scene, 'highres_depth')):
#         print(sample)
#         # if gt depth missing points are less than 20% of the image
#         depth_path = os.path.join(arkitScenes_path, 'Training', scene, 'highres_depth', sample)
#         depth = np.array(Image.open(depth_path))
#         missing = np.sum(depth == 0)
#         # if missing / depth.size < 0.2:
#             with open('arkit_split_training.txt', 'a') as f:
#                 f.write(os.path.join(scene, 'lowres_wide', sample) + '\n')

## Matterport3D dataset

In [2]:
test_files_path = './Matterport3D/test_file_list_normal.txt'
train_files_path = './Matterport3D/train_file_list_normal.txt'

Matterport3D = '/mnt/IRONWOLF2/yiqinzhao/Matterport3D/v1/scans/'

In [20]:
def create_split(file, split):
    # Initialize dictionaries to store color and depth paths by ID
    color_images = {}
    depth_images = {}

    # Read the file and populate the dictionaries
    with open(file, 'r') as input_file:
        for line in input_file:
            parts = line.strip().split(' ')
            # Original paths
            color_path = parts[0].replace('/path/to/your/', '')
            depth_path = parts[1].replace('/path/to/your/', '')

            # Split paths to reformat them
            color_parts = color_path.split('/')
            depth_parts = depth_path.split('/')
            
            

            # Reformat paths by moving the ID to the front and replacing folder names
            new_color_path = f"{color_parts[1]}/undistorted_color_images/{'/'.join(color_parts[2:])}"
            new_depth_path = f"{depth_parts[1]}/undistorted_depth_images/{'/'.join(depth_parts[2:])}"

            # Extract IDs
            color_id = color_parts[1]
            depth_id = depth_parts[1]
            

            # Append paths to their respective lists within the dictionaries
            if color_id not in color_images:
                color_images[color_id] = []
            color_images[color_id].append(new_color_path)
            
            if depth_id not in depth_images:
                depth_images[depth_id] = []
            depth_images[depth_id].append(new_depth_path)
            
            depth_path = os.path.join(Matterport3D, 'Training', scene, 'highres_depth', sample)
            depth = np.array(Image.open(depth_path))
            missing = np.sum(depth == 0)

    # Write the matched paths to the output file, pairing each color with its corresponding depth image
    output_file_path = f'./Matterport3D_{split}_split.txt'
    with open(output_file_path, 'w') as output_file:
        for id in color_images:
            if id in depth_images:
                # Iterate through both lists and pair images by their index
                for color_path, depth_path in zip(color_images[id], depth_images[id]):
                    # if gt depth missing points are less than 20% of the image
                #   depth_path = os.path.join(arkitScenes_path, 'Validation', scene, 'highres_depth', sample)
        #         depth = np.array(Image.open(depth_path))
        #         missing = np.sum(depth == 0)
        #         if missing / depth.size < 0.2:
        #             with open('arkit_split_validation.txt', 'a') as f:
        #                 f.write(os.path.join(scene, 'lowres_wide', sample) + '\n')
                    output_file.write(f"{color_path} {depth_path}\n")

    return output_file_path


In [ ]:
create_split(test_files_path, 'test')

In [ ]:

create_split(train_files_path, 'train')